### House Price Prediction with MLflow
1. Run the hyperparameter tuning while training the model
2. Log every hyperparameter and metrics
4. Compare the  results of the various runs
5. Choose the best model and  register

In [1]:
import pandas as pd
import mlflow 
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [4]:
## preparing the dataset
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price']=housing.target
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


#### Train test split, Model Hyperparameter tuning and model training

In [5]:
from urllib.parse import urlparse

## independent and dependent features
X=data.drop('Price',axis=1)
y=data['Price']

## splitting the data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) 

from mlflow.models import infer_signature
signature=infer_signature(X_train,y_train)

In [6]:
## define the hyperparameters
param_grid = {
    'n_estimators': [10, 30, 50, 70],
    'max_depth': [2, 4, 6],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
}

In [8]:
## Hyperparameter tuning using GridSearchCV
 
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    clf=GridSearchCV(rf,param_grid,cv=5)
    clf.fit(X_train,y_train)
    print("Best hyperparameters: ", clf.best_params_)
    return clf

In [11]:
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_squared_error

# Start the MLFlow run
with mlflow.start_run():
    ## Perform hyperparameter tuning
    clf = hyperparameter_tuning(X_train, y_train, param_grid)

    ## Get the best model
    best_model = clf.best_estimator_

    ## Evaluate the model
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    ## Log the best parameters and metrics
    mlflow.log_param("n_estimators", clf.best_params_['n_estimators'])
    mlflow.log_param("max_depth", clf.best_params_['max_depth'])
    mlflow.log_param("min_samples_split", clf.best_params_['min_samples_split'])
    mlflow.log_param("min_samples_leaf", clf.best_params_['min_samples_leaf'])
    mlflow.log_metric("MSE", mse)

    ## Set tracking URI
    tracking_uri = "http://localhost:5000"
    mlflow.set_tracking_uri(tracking_uri)
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="RandomForestRegressor")
    else:
        mlflow.sklearn.log_model(best_model, "model")


Best hyperparameters:  {'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 70}


2025/03/08 13:20:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'RandomForestRegressor'.
2025/03/08 13:20:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestRegressor, version 1


🏃 View run lyrical-lark-60 at: http://localhost:5000/#/experiments/0/runs/98cf2c60a42a4f10a0fe38b5ac47dd7a
🧪 View experiment at: http://localhost:5000/#/experiments/0


Created version '1' of model 'RandomForestRegressor'.
